In [1]:
import os
from os import path as op
import geopandas as gdp
import geemap
import ee

# Making module or packages collection attribute became callable..
import collections
collections.Callable = collections.abc.Callable

#plotting packages
import pandas as pd#Used for data analysis also include DataFrame df data structure
import matplotlib.pyplot as plt#For plotting of dataframes
import numpy as np
import altair as alt#declarative visualization(Used for charting)

import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

from geemap import chart #for chart plotting


#packages used in database connection.
import pandas.io.sql as sqlio
import psycopg2 as ps

import warnings
warnings.filterwarnings("ignore")
Map=geemap.Map()



SMALL SCALE CROP MAPPING

In [2]:
# Connection to Crop_Mapping db(iggres_Database)
connection=ps.connect(dbname="crop_mapping",#Database_name.
                 user="postgres",#Database_user.
                 password=".Xy!@D3K_3ev3R.2!jL!",#Database_passcode.
                 host="iggresserver.dkut.ac.ke",#Db Host
#                  port="5432"#Db operational port
                )

# print("Connection to crop_mapping Database established Successful")

In [3]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

admin1_widget = widgets.Text(
    description='Project_Name:',
    value='enter your project',
    width=200,
    style=style
)

# admin2_widget = widgets.Text(
#     description='DATE:',
#     value='enter start date',
#     width=300,
#     style=style
# )

# admin3_widget = widgets.Text(
#     description='DATE:',
#     value='enter end date',
#     width=300,
#     style=style
# )
Data_collection = widgets.Button(
    description='select your data',
    button_style='success',
    tooltip='Click me',
    style=style
)
AOI = widgets.Button(
    description='Drawn study area',
    button_style='success',
    tooltip='Click me',
    style=style
)
load_model = widgets.Button(
    description='Load Model',
    button_style='danger',
    tooltip='Click me',
    style=style
)
submit = widgets.Button(
    description='Select Training points',
    button_style='primary',
    tooltip='Click me',
    style=style
)

classification = widgets.Button(
    description='Classification results',
    button_style='primary',
    tooltip='Click me',
    style=style
)

graphing_NDVI= widgets.Button(
    description='Crop health',
    button_style='danger',
    tooltip='Click me',
    style=style
)
Area= widgets.Button(
    description='Crop_Area',
    button_style='primary',
    tooltip='Click me',
    style=style
)
Crop_yield= widgets.Button(
    description='Crop_Yield',
    button_style='success',
    tooltip='Click me',
    style=style
)
widgets.VBox([
    widgets.HBox([admin1_widget]),
#     Data_collection,
    submit,
#     Crop_health,
    AOI,
    load_model,
    classification,
    graphing_NDVI,
    Area,
    Crop_yield
   
])


In [5]:
# Map

In [6]:
def create_reduce_region_function(geometry,#The region that reduction occurs
                                  reducer=ee.Reducer.mean(),#Method of reduction used i.e mean
                                  scale=1000,# number that defines the nominal scale in meters of the projection to work in.
                                  crs='EPSG:4326',#Coordiante Reference system used for projection or reprojection.
                                  bestEffort=True,
                                  maxPixels=1e13,# A number specifying the maximum number of pixels to reduce.
                                  tileScale=4):#Representation of the scaling factor used to reduce aggregation tile size


  def reduce_region_function(img):
    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')
  return ee.Dictionary.fromLists(prop_names, prop_lists)
# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year#Add year timestamp to the dataframe
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month#Add month timestamp to the dataframe
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day#Add day timestamp to the dataframe
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear#Add Day of year timestamp to the dataframe
  return df#return function

In [7]:
# Click event handler

def select_data_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        admin1_id = admin1_widget.value
        # Connection to Crop_Mapping db(iggres_Database)
        connection=ps.connect(dbname="crop_mapping",#Database_name.
                 user="postgres",#Database_user.
                 password=".Xy!@D3K_3ev3R.2!jL!",#Database_passcode.
                 host="iggresserver.dkut.ac.ke",#Db Host
                 port="5432"#Db operational port
                )
        sql="""select*FROM crop_mapping
        WHERE variety= 'Nakuru';"""
           
Data_collection.on_click(select_data_clicked)



In [8]:
# Click event handler

def submit_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        admin1_id = admin1_widget.value
           
        sql="""select*FROM crop_mapping
        WHERE variety= 'Nakuru';"""
        gdf = geemap.read_postgis(sql, connection)#Crop_mapping data based on connection to KSA DB
        gdf.crs = {'init': 'epsg:4326'} #Reprojection of training points or polygon into unique reference code i.e 4326.
        global fc
        fc = geemap.gdf_to_ee(gdf)#FROM GDF TO ee feature collection
        Map.addLayer(fc, {}, "points from db")#Add feature collection features to the layers.
        Map.centerObject(fc, 12)#Center the map view to feature collection
           
submit.on_click(submit_clicked)



In [9]:
# Click event handler

def AOI_clicked(b):
    
    with output_widget:
        global muringato1
        muringato1=Map.user_rois.geometry()
        Map.addLayer(muringato1,{},"muringato")
#         print("Successfully drawn Area of Intrest")
           
AOI.on_click(AOI_clicked)



In [10]:
# Click event handler



def load_model_clicked(b):
     with output_widget:
        start_date="2020-01-01"#Set start_date(yy/mon/day)
        end_date="2020-03-31"#Set End_date(yy/mon/day)
        season = ee.Filter.date(start_date,end_date);#Filter image based on the time frame(start_date and end_date)
#         muringato=Map.user_rois.geometry()

#-------------SENTINEL 1(SAR) DATA------------------#
        sentinel_1= ee.ImageCollection('COPERNICUS/S1_GRD');
        sCollection=sentinel_1\
       .filterBounds(muringato1) \
       .filter(season) \
       .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
       .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
       .filter(ee.Filter.eq('instrumentMode', 'IW'))
        desc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));
        asc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));
#         Map.addLayer(sCollection,{},"test_sentinel")
        global composite
        composite = ee.Image.cat([
        asc.select('VH').mean(),
        asc.select('VV').mean(),
        desc.select('VH').mean()
       ]).focal_median();
    #################################
     #-------------SENTINEL_2A DATA------------------#  
        global image
        image = ee.ImageCollection('COPERNICUS/S2')\
        .filterBounds(muringato1)\
        .filterDate("2020-01-01","2020-03-31")\
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",12))\
        .median()\
        .select('B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B10', 'B11')\
        .clip(muringato1)

        sentinel_2Avispar={"min":0, "max":2500,"bands": ['B4','B3','B2']}#Visualization parameters used.
        Map.addLayer(image,sentinel_2Avispar, 'Sentinel-2')#Add sentinel_2A to the layer 

        
load_model.on_click(load_model_clicked)






In [11]:
# Click event handler

def NDVI_clicked(b):
    
    with output_widget:
#         Map= geemap.Map()
#         ndvi_map.centerObject(muringato)
#NDVI( Normalized Difference Vegetation Index )
        NDVI = image.normalizedDifference(['B8', 'B4'])#normalized difference is computed as (first − second) / (first + second).
        ndvivis_parametres = {'min':0, 'max':1, 'palette': ['red','brown','yellow', 'green'] }#NDVI visualization parameters
        Map.addLayer(NDVI, ndvivis_parametres, 'NDVI')#Add Normalized Difference Vegetation Index to the layers

#Visualizaation Legend:


        vis_params = {
    'min': 0,
    'max': 1,
    'palette':['e82d09', '934d21', 'e8dc10', '53ea0e'],
}

        colors = vis_params['palette']
        vmin = vis_params['min']
        vmax = vis_params['max']

        Map.add_colorbar(vis_params,label='NDVI Readings')

        Map.addLayerControl()

        legend_dict = {

       'non-plants(0 to 0.18)': 'e82d09',
       'Uhealth_Plants(0.18 to 0.41)': '934d21',
       'Moderatly_health_Plants(0.41 to 0.0.69)': 'e8dc10',
       'Very_health(0.69 to 1)': '53ea0e',}

        Map.add_legend(legend_title="NDVI_readings", legend_dict=legend_dict)

        Map
graphing_NDVI.on_click(NDVI_clicked)






In [12]:
def classification_image(b):
    
    with output_widget:
        
        Fused_images = image.addBands(composite)#Creation of composite image of sentinel 1 and sentinel 2A
#Visualiztion parameters for fused 
        fused_vis = {
        'min':0,
        'max'  : 2500,
        'bands' : ['B2', 'B3','B4']#Band selection.
    }
        training = ee.FeatureCollection(fc)

#Create a buffer:
        training1 = training.map(lambda f: f.buffer(5))
# Map.addLayer(training1,{},"buffer")
        label = 'landcover'#Unique property(cropland) with different unique values..
        bands=["B4","B3","B2","VV","VH"]
        global Input
        Input=Fused_images.select(bands)
        trainImage=Input.sampleRegions(**{
    'collection':training,
    'properties':[label],
    'scale':30
})

        trainingData=trainImage.randomColumn()

        trainSet=trainingData.filter(ee.Filter.lessThan('random',0.75))
        testdata=trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.75))


#visualization parameters applied on the Random forest classifier

        init_params = {"numberOfTrees":10, # the number of individual decision tree models
              "variablesPerSplit":None,  # the number of features to use per split
              "minLeafPopulation":1, # smallest sample size possible per leaf
              "bagFraction":0.5, # fraction of data to include for each individual tree model
              "maxNodes":None, # max number of leafs/nodes per tree
               "seed":0}  # random seed for "random" choices like sampling. Setting this allows others to reproduce your exact results even with stocastic parameters
        global classifier

        classifier = ee.Classifier.smileRandomForest(**init_params).train(trainImage, label, bands)
# Classify the image.

#Application of the random forest classifier for the purpose of image classification.
        global classified
        classified=Input.classify(classifier)
        palette = [
  '#FFFF00', #Cropland_Maize(1)--Yellow
  '#83b282',#Cropland_coffee(2)--light green
  '#fadadd',#Cropland_Napier(3)--Pale Pink
  '#964B00', #Cropland_Tea(4)--Brown
  '#8B8000',#lightgreen(5) Cropland_Banana(5)--Dark Yellow
  '#FFC0CB',#Beans(6)-- Pink
  '#FF0000',#Vegetable(7)--red
  '#FFFF00', #Cropland_Maize(8)--Yellow
  '#83b282',#Cropland_coffee(9)--light green
  '#fadadd',#Cropland_Napier(10)--Pale Pink
  '#964B00', #Cropland_Tea(11)--Brown
  '#8B8000',#lightgreen(12) Cropland_Banana(5)--Dark Yellow
  '#FFC0CB',#Beans(13)-- Pink
  '#FF0000',#Vegetable(14)--red
  '#83b282',#Cropland_coffee(15)--light green
  '#fadadd',#Cropland_Napier(16)--Pale Pink
  '#964B00', #Cropland_Tea(17)--Brown
  '#8B8000',#lightgreen(18) Cropland_Banana(5)--Dark Yellow
  '#FFC0CB',#Beans(19)-- Pink  
 ];
        Map.addLayer(classified,
            {'min': 0, 'max': 39, 'palette': palette},
             'classification')


        legend_dict = {
    'Beans': 'brown',
    'Coffee': 'd1def8',
    'Maize': 'dec5c5',
    'Napier': 'd99282',
    'Potato': 'eb0000',
    'Tea': 'ab0000',
    'Vegetable': 'b3ac9f',
    'Banana': '68ab5f',
    'Peas': '1c5f2c'}
    Map.add_legend(legend_title="  Crop_Classification", legend_dict=legend_dict)


classification.on_click(classification_image)

In [13]:
# # Click event handler

# def select_data_clicked(b):
    
#     with output_widget:
#         output_widget.clear_output()
#         admin1_id = admin1_widget.value
#         # Connection to Crop_Mapping db(iggres_Database)
#         connection=ps.connect(dbname="crop_mapping",#Database_name.
#                  user="postgres",#Database_user.
#                  password=".Xy!@D3K_3ev3R.2!jL!",#Database_passcode.
#                  host="iggresserver.dkut.ac.ke",#Db Host
#                  port="5432"#Db operational port
#                 )
#         sql="""select*FROM crop_mapping
#         WHERE variety= 'Nakuru';"""
        
        
        
#         Total_studyArea= classified.geometry().area()
#         Total_AreaSqKm = ee.Number(Total_studyArea).divide(1e6).round()

# #ESTIMATED CROP AREA


# # Used Absent used to show points that were not picked using the D_Kcrop mobile _app
#         names = ['Maize','Forest','Napier','Arrow-roots','Tea','Bananas',
#         'Absent---7', 'Absent----8', 'Sorghum', 'Absent----10',
#          'Absent----11', 'Absent---12',
#           'onions', 'Absent--14', 'Kales', 'Potato', 'Absent--17', 'Coffee','sweet-potatoes']

#         count = classified.eq([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])#


#         total = count.multiply(ee.Image.pixelArea()).divide(4046.86).rename(names)#mutiply and divide the area with 1e6 to conv into km

#         area = total.reduceRegion(**{
#   'reducer': ee.Reducer.sum(),
#   'geometry': muringato1,
#   'scale': 30,
#   'maxPixels': 1e9
#            })
#         print(area,"Area in square Acres:")
#         area.getInfo()

           
# Area.on_click(select_data_clicked)

